In [1]:
!pip install tqdm
!pip install pytorch-pretrained-bert
!pip install keras
!pip install tensorflow

fastai 1.0.40 requires dataclasses, which is not installed.
fastai 1.0.40 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 122kB 33.3MB/s 
fastai 1.0.40 requires dataclasses, which is not installed.
fastai 1.0.40 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 317kB 30.7MB/s 
    100% |████████████████████████████████| 61kB 27.4MB/s 
    100% |████████████████████████████████| 61kB 28.3MB/s 
fastai 1.0.40 re

In [19]:
import logging
import os
import argparse
import random
from tqdm import tqdm, trange
import csv
import pandas as pd
import re

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForQuestionAnswering,BertForMultipleChoice
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


In [3]:
train = pd.read_csv("InferSent/encoder/data/train.tsv",sep='\t')
train.head(3)

,ID,Question Stem,Choices,Complete Question,Answer Key
0,7-980,The sun is responsible for,(A) puppies learning new tricks (B) children g...,The sun is responsible for (A) puppies learnin...,D
1,7-584,When standing miles away from Mount Rushmore,(A) the mountains seem very close (B) the moun...,When standing miles away from Mount Rushmore (...,D
2,7-870,When food is reduced in the stomach,(A) the mind needs time to digest (B) take a s...,When food is reduced in the stomach (A) the mi...,C


In [4]:
#Create new features from the answer choices  
for index, row in train.iterrows():
    ans = re.split("(\([A-Z]\))", row['Choices'])
    ans.remove('')
    train.set_value(index,'OptionA',ans[1])
    train.set_value(index,'OptionB',ans[3])
    train.set_value(index,'OptionC',ans[5])
    train.set_value(index,'OptionD',ans[7])
    
#Find the answer key and replace it with the value   
for index, row in train.iterrows():
    ans = re.split("(\([A-Z]\))", row['Choices'])
    ans.remove('')
    if row['Answer Key'] == 'A':
        train.set_value(index,'Result',ans[1])
    elif row['Answer Key'] == 'B':
        train.set_value(index,'Result',ans[3])
    elif row['Answer Key'] == 'C':
        train.set_value(index,'Result',ans[5])
    elif row['Answer Key'] == 'D':
        train.set_value(index,'Result',ans[7]) 

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: FutureWarning: set_value

In [5]:
num_of_categories = 4957
shuffled = train.reindex(np.random.permutation(train.index))
a = shuffled[shuffled['Answer Key'] == 'A'][:num_of_categories]
b = shuffled[shuffled['Answer Key'] == 'B'][:num_of_categories]
c = shuffled[shuffled['Answer Key'] == 'C'][:num_of_categories]
d = shuffled[shuffled['Answer Key'] == 'D'][:num_of_categories]
concated = pd.concat([a,b,c,d], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

In [6]:
from keras.utils.np_utils import to_categorical
#One-hot encode the labels
concated.loc[concated['Answer Key'] == 'A', 'LABEL'] = 0
concated.loc[concated['Answer Key'] == 'B', 'LABEL'] = 1
concated.loc[concated['Answer Key'] == 'C','LABEL'] = 2
concated.loc[concated['Answer Key'] == 'D','LABEL'] = 3
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=4)
print(labels[:10])
if 'CATEGORY' in concated.keys():
    concated.drop(['CATEGORY'], axis=1)    

Using TensorFlow backend.


3364    2
99      0
3163    2
4315    3
1090    0
4410    3
1301    0
2497    1
1       0
1368    0
Name: LABEL, dtype: int64
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [7]:
concated.head(10)

,ID,Question Stem,Choices,Complete Question,Answer Key,OptionA,OptionB,OptionC,OptionD,Result,LABEL
3364,14-1417,"As altitude increases, the air temperature wil...",(A) rise (B) go up (C) nosedive (D) elevate,"As altitude increases, the air temperature wil...",C,rise,go up,nosedive,elevate,nosedive,2
99,13-620,An example of evaporation is a body of water d...,(A) prolonged solar expsosure (B) cooking lunc...,An example of evaporation is a body of water d...,A,prolonged solar expsosure,cooking lunch,taking naps,smiting enemies,prolonged solar expsosure,0
3163,12-282,migration is when animals travel from the nort...,(A) beach area (B) spring (C) winter (D) birds,migration is when animals travel from the nort...,C,beach area,spring,winter,birds,winter,2
4315,14-374,"Aside from plants, lizards also eat bugs for",(A) singing (B) crying (C) nothing (D) sustenance,"Aside from plants, lizards also eat bugs for (...",D,singing,crying,nothing,sustenance,sustenance,3
1090,8-275,Puerto Rico experienced many landslides due to...,(A) 2017 (B) 2015 (C) 2012 (D) 2009,Puerto Rico experienced many landslides due to...,A,2017,2015,2012,2009,2017,0
4410,13-45,What most likely caused the castle built by ch...,(A) a cat ran off with it (B) a dog carried it...,What most likely caused the castle built by ch...,D,a cat ran off with it,a dog carried it away in its mouth,someone cast a magic spell on it,very strong eddies of fast moving air carried...,very strong eddies of fast moving air carried...,3
1301,10-257,If an object is close then how will that objec...,(A) gigantic (B) minute (C) small (D) tiny,If an object is close then how will that objec...,A,gigantic,minute,small,tiny,gigantic,0
2497,9-939,Humans go through multiple stages in their lif...,(A) Stage 7 (B) being an adult (C) high school...,Humans go through multiple stages in their lif...,B,Stage 7,being an adult,high school,puberty,being an adult,1
1,11-694,A wet globe that is enormous in mass is likely...,(A) layer (B) sun (C) brain (D) labyrinth,A wet globe that is enormous in mass is likely...,A,layer,sun,brain,labyrinth,layer,0
1368,12-1161,What would have a better chance of survival if...,(A) Hamsters (B) Snakes (C) Killer Whales (D) ...,What would have a better chance of survival if...,A,Hamsters,Snakes,Killer Whales,Tigers,Hamsters,0


In [8]:
sentences = []
with open('InferSent/encoder/data/openbook.txt') as f:
    for line in f:
        sentences.append(line.strip())
print(len(sentences))

1326


In [9]:
class QAExample(object):
    """A single training/test example for the SWAG dataset."""
    def __init__(self,
                 QA_id,
                 context_sentence,
                 question_stem,
                 ending_0,
                 ending_1,
                 ending_2,
                 ending_3,
                 label = None):
        self.qa_id = QA_id
        self.context_sentence = context_sentence
        self.question_stem = question_stem
        self.endings = [
            ending_0,
            ending_1,
            ending_2,
            ending_3,
        ]
        self.label = label

class InputFeatures(object):
    def __init__(self,
                 example_id,
                 choices_features,
                 label

    ):
        self.example_id = example_id
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label        

In [10]:

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

def select_field(features, field):
    return [
        [
            choice[field]
            for choice in feature.choices_features
        ]
        for feature in features
    ]

def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x            

In [11]:
 examples = [
        QAExample(
            QA_id = concated['ID'].values,
            context_sentence = sentences,
            question_stem = concated["Question Stem"].values,
            ending_0 = concated['OptionA'].values,
            ending_1 = concated['OptionB'].values,
            ending_2 = concated['OptionC'].values,
            ending_3 = concated['OptionD'].values,
            label = concated["LABEL"].values
        )
    ]

In [12]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


03/28/2019 15:52:19 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpyyfou15z
100%|██████████| 231508/231508 [00:00<00:00, 761660.39B/s]
03/28/2019 15:52:20 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmpyyfou15z to cache at /home/ec2-user/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/28/2019 15:52:20 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /home/ec2-user/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/28/2019 15:52:20 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmpyyfou15z
03/28/2019 15:52:20 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file htt

In [13]:
from keras.preprocessing.sequence import pad_sequences
features = []
max_seq_length = 80
for example_index, example in enumerate(examples):
    
    # tokenise the questions
    context_sentence_tokens = tokenizer.tokenize(''.join(example.question_stem))
  
        
    choices_features = [] 
    for ending_index, ending in enumerate(example.endings):
        
        #Select all the question tokens
        context_tokens_sentence = context_sentence_tokens[:]
        
        #Select all the answer tokens
        ending_tokens = tokenizer.tokenize(''.join(ending))
        
        _truncate_seq_pair(context_tokens_sentence, ending_tokens, max_seq_length - 3)
        
        
        tokens = ["[CLS]"] + context_tokens_sentence + ["[SEP]"] + ending_tokens + ["[SEP]"]
                                                          
                                                                  
        segment_ids = [0] * (len(context_tokens_sentence) + 2) + [1] * (len(ending_tokens) + 1)   
     
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
      
        input_mask = [1] * len(input_ids)
      

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        choices_features.append((tokens, input_ids, input_mask, segment_ids))
        
    for l in example.label:                                                              
    # get the labels
        labels = l 
                            
        features.append(
            InputFeatures(
                example_id = example.qa_id,
                choices_features = choices_features,
                label = labels
            )
        )  


In [14]:
# for example_index, example in enumerate(features):
#     print(example.label)

In [15]:
import os

local_rank = -1
fp16 = ""
no_cuda = ""
gradient_accumulation_steps = 4
seed = 42
train_batch_size = 16
do_train=True
do_eval=True
num_train_epochs = 3
bert_model = 'bert-base-uncased'
learning_rate = 2e-5
warmup_proportion = 0.1
loss_scale = 0
train_features = features
output_dir=os.getcwd() + "models"

In [16]:
if  local_rank == -1 or no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
else:
    torch.cuda.set_device(local_rank)
    device = torch.device("cuda", local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
    device, n_gpu, bool(local_rank != -1), fp16))

if gradient_accumulation_steps < 1:
    raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                        gradient_accumulation_steps))

train_batch_size = int(train_batch_size / gradient_accumulation_steps)

03/28/2019 15:52:40 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: 


In [20]:

model = BertForMultipleChoice.from_pretrained('bert-base-uncased', num_choices=4)
if fp16:
    model.half()
model.to(device)
if local_rank != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    model = DDP(model)
elif n_gpu > 1:
    model = torch.nn.DataParallel(model)

03/28/2019 15:54:00 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmp4ah34soq
100%|██████████| 407873900/407873900 [00:22<00:00, 18121951.75B/s]
03/28/2019 15:54:23 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmp4ah34soq to cache at /home/ec2-user/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/28/2019 15:54:24 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /home/ec2-user/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/28/2019 15:54:24 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmp4ah34soq
03/28/2019 15:54:24 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https

In [21]:
len(train)

4957

In [22]:
#train_examples = None
num_train_steps = None
if do_train:
    num_train_steps = int(len(train) / train_batch_size / gradient_accumulation_steps * num_train_epochs)

In [23]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
t_total = num_train_steps

if local_rank != -1:
    t_total = t_total // torch.distributed.get_world_size()
if fp16:
    try:
        from apex.optimizers import FP16_Optimizer
        from apex.optimizers import FusedAdam
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    optimizer = FusedAdam(optimizer_grouped_parameters,
                          lr=learning_rate,
                          bias_correction=False,
                          max_grad_norm=1.0)
    if loss_scale == 0:
        optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
    else:
        optimizer = FP16_Optimizer(optimizer, static_loss_scale=loss_scale)
else:
    optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=learning_rate,
                     warmup=warmup_proportion,
                     t_total=t_total)  

input_ids = select_field(train_features, 'input_ids')
flat_list_input_ids = []
for item in input_ids:
    flat_list_input_ids.append(item)

input_mask = select_field(train_features, 'input_mask')
flat_list_input_mask = []
for item in input_mask:
    flat_list_input_mask.append(item)


segment_ids = select_field(train_features, 'segment_ids')
flat_list_segment_ids = []
for item in segment_ids:
    flat_list_segment_ids.append(item)

# labels_y=[]
# for item in train_features:
#     for label in item.label:
#         labels_y.append(label)

global_step = 0
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)

#tags = pad_sequences([f.label for f in train_features],
#                     maxlen=10, padding="post", truncating="post")

tags = [f.label for f in train_features]


all_input_ids = torch.tensor(input_ids, dtype=torch.long)
all_segment_ids = torch.tensor(segment_ids, dtype=torch.long)
all_input_mask = torch.tensor(input_mask, dtype=torch.long)
all_label = torch.tensor(tags ,dtype=torch.long)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,all_label)

all_input_ids.size(),all_input_mask.size(),all_segment_ids.size(),all_label.size()


03/28/2019 15:56:59 - INFO - __main__ -   ***** Running training *****
03/28/2019 15:56:59 - INFO - __main__ -     Num examples = 4957
03/28/2019 15:56:59 - INFO - __main__ -     Batch size = 4
03/28/2019 15:56:59 - INFO - __main__ -     Num steps = 929


(torch.Size([4957, 4, 80]),
 torch.Size([4957, 4, 80]),
 torch.Size([4957, 4, 80]),
 torch.Size([4957]))

In [24]:
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

In [25]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,all_label)


for _ in trange(3, desc="Epoch"):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        print(input_ids.shape,input_mask.shape,segment_ids.shape,label_ids.shape)
        loss = model(input_ids, input_mask,segment_ids,label_ids)


        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.

        if fp16 and loss_scale != 1.0:
            # rescale loss for fp16 training
            # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
            loss = loss * loss_scale

        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
            
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        if fp16:
            optimizer.backward(loss)
        else:
            loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            # modify learning rate with special warm up BERT uses
            lr_this_step = learning_rate * warmup_linear(global_step/t_total, warmup_proportion)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_this_step
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Iteration:   0%|          | 0/1240 [00:00<?, ?it/s]

torch.Size([4, 4, 80]) torch.Size([4, 4, 80]) torch.Size([4, 4, 80]) torch.Size([4])



Iteration:   0%|          | 1/1240 [00:01<28:35,  1.38s/it]

torch.Size([4, 4, 80]) torch.Size([4, 4, 80]) torch.Size([4, 4, 80]) torch.Size([4])



Iteration:   0%|          | 2/1240 [00:01<23:01,  1.12s/it]

...


Iteration: 100%|█████████▉| 1238/1240 [10:12<00:00,  2.05it/s]

torch.Size([4, 4, 80]) torch.Size([4, 4, 80]) torch.Size([4, 4, 80]) torch.Size([4])



Iteration: 100%|█████████▉| 1239/1240 [10:12<00:00,  2.08it/s]

torch.Size([1, 4, 80]) torch.Size([1, 4, 80]) torch.Size([1, 4, 80]) torch.Size([1])



Epoch: 100%|██████████| 3/3 [30:42<00:00, 614.55s/it]


In [26]:
output_dir

'/home/ec2-user/SageMaker/Thesismodels'

In [27]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
output_model_file = os.path.join(output_dir, "pytorch_model.bin")
if do_train:
    torch.save(model_to_save.state_dict(), output_model_file)

In [28]:
model_state_dict = torch.load(output_model_file)
model_saved = BertForMultipleChoice.from_pretrained(bert_model, state_dict=model_state_dict, num_choices=4)
model_saved.to(device)

03/28/2019 16:29:14 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/ec2-user/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/28/2019 16:29:14 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /home/ec2-user/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpap7z5opl
03/28/2019 16:29:19 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size"

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
 

In [29]:
test = pd.read_csv("InferSent/encoder/data/test.tsv",sep='\t')
test.shape

(500, 5)

In [30]:
#Create new features from the answer choices  
for index, row in test.iterrows():
    ans = re.split("(\([A-Z]\))", row['Choices'])
    ans.remove('')
    test.set_value(index,'OptionA',ans[1])
    test.set_value(index,'OptionB',ans[3])
    test.set_value(index,'OptionC',ans[5])
    test.set_value(index,'OptionD',ans[7])
    
#Find the answer key and replace it with the value   
for index, row in test.iterrows():
    ans = re.split("(\([A-Z]\))", row['Choices'])
    ans.remove('')
    if row['Answer Key'] == 'A':
        test.set_value(index,'Result',ans[1])
    elif row['Answer Key'] == 'B':
        test.set_value(index,'Result',ans[3])
    elif row['Answer Key'] == 'C':
        test.set_value(index,'Result',ans[5])
    elif row['Answer Key'] == 'D':
        test.set_value(index,'Result',ans[7]) 

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:17: FutureWarning: set_value

In [32]:
num_of_categories_test = 500
shuffled_test = test.reindex(np.random.permutation(test.index))

a_test = shuffled_test[shuffled_test['Answer Key'] == 'A'][:num_of_categories_test]
b_test = shuffled_test[shuffled_test['Answer Key'] == 'B'][:num_of_categories_test]
c_test = shuffled_test[shuffled_test['Answer Key'] == 'C'][:num_of_categories_test]
d_test = shuffled_test[shuffled_test['Answer Key'] == 'D'][:num_of_categories_test]

concated_test = pd.concat([a_test,b_test,c_test,d_test], ignore_index=True)
print(len(a_test),len(b_test),len(c_test),len(d_test))

#Shuffle the dataset
concated_test = concated_test.reindex(np.random.permutation(concated_test.index))
concated_test['LABEL'] = 0


138 126 132 104


In [33]:
from keras.utils.np_utils import to_categorical
#One-hot encode the labels
concated_test.loc[concated_test['Answer Key'] == 'A', 'LABEL'] = 0
concated_test.loc[concated_test['Answer Key'] == 'B', 'LABEL'] = 1
concated_test.loc[concated_test['Answer Key'] == 'C','LABEL'] = 2
concated_test.loc[concated_test['Answer Key'] == 'D','LABEL'] = 3
print(concated_test['LABEL'][:10])
labels_test = to_categorical(concated_test['LABEL'], num_classes=4)
print(labels_test[:10])
if 'CATEGORY' in concated_test.keys():
    concated_test.drop(['CATEGORY'], axis=1)   

482    3
18     0
425    3
119    0
400    3
275    2
5      0
434    3
141    1
67     0
Name: LABEL, dtype: int64
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [34]:
 test_examples = [
        QAExample(
            QA_id = concated_test['ID'].values,
            context_sentence = sentences,
            question_stem = concated_test["Question Stem"].values,
            ending_0 = concated_test['OptionA'].values,
            ending_1 = concated_test['OptionB'].values,
            ending_2 = concated_test['OptionC'].values,
            ending_3 = concated_test['OptionD'].values,
            label = concated_test["LABEL"].values
        )
    ]

In [35]:
concated_test["LABEL"].shape

(500,)

In [36]:
from keras.preprocessing.sequence import pad_sequences
features = []
max_seq_length = 16
for example_index, example in enumerate(test_examples):
    
    # tokenise the questions
    context_sentence_tokens = tokenizer.tokenize(''.join(example.question_stem))
      
        
    test_choices_features = [] 
    for ending_index, ending in enumerate(example.endings):
        
        #Select all the question tokens
        context_tokens_sentence = context_sentence_tokens[:]
        
        #Select all the answer tokens
        ending_tokens = tokenizer.tokenize(''.join(ending))
        
        _truncate_seq_pair(context_tokens_sentence, ending_tokens, max_seq_length - 3)   
        tokens = ["[CLS]"] + context_tokens_sentence + ["[SEP]"] + ending_tokens + ["[SEP]"]                                          
        segment_ids = [0] * (len(context_tokens_sentence) + 2) + [1] * (len(ending_tokens) + 1)   
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
      

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        test_choices_features.append((tokens, input_ids, input_mask, segment_ids))
        
    for l in example.label:                                                              
    # get the labels
        labels = l 
                            
        features.append(
            InputFeatures(
                example_id = example.qa_id,
                choices_features = test_choices_features,
                label = labels
            )
        )  

In [37]:
test_features = features
input_ids = select_field(test_features, 'input_ids')
input_mask = select_field(test_features, 'input_mask')
segment_ids = select_field(test_features, 'segment_ids')


global_step = 0
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)

#tags = pad_sequences([f.label for f in train_features],
#                     maxlen=10, padding="post", truncating="post")

tags = [f.label for f in test_features]


all_input_ids = torch.tensor(input_ids, dtype=torch.long)
all_segment_ids = torch.tensor(segment_ids, dtype=torch.long)
all_input_mask = torch.tensor(input_mask, dtype=torch.long)
all_label = torch.tensor(tags ,dtype=torch.long)
test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,all_label)

all_input_ids.size(),all_input_mask.size(),all_segment_ids.size(),all_label.size()

03/28/2019 16:30:25 - INFO - __main__ -   ***** Running training *****
03/28/2019 16:30:25 - INFO - __main__ -     Num examples = 4957
03/28/2019 16:30:25 - INFO - __main__ -     Batch size = 4
03/28/2019 16:30:25 - INFO - __main__ -     Num steps = 929


(torch.Size([500, 4, 16]),
 torch.Size([500, 4, 16]),
 torch.Size([500, 4, 16]),
 torch.Size([500]))

In [38]:
# Run prediction for full data
predict_batch_size = 24
eval_sampler = SequentialSampler(test_data)
eval_dataloader = DataLoader(test_data, sampler=eval_sampler, batch_size=predict_batch_size)

In [39]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

In [40]:

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
        logits = model(input_ids, segment_ids, input_mask)

    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += input_ids.size(0)
    nb_eval_steps += 1

eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / nb_eval_examples

result = {'eval_loss': eval_loss,
          'eval_accuracy': eval_accuracy,
          'global_step': global_step,
          'loss': tr_loss/nb_tr_steps}

In [41]:
result

{'eval_loss': 1.3894545237223308,
 'eval_accuracy': 0.246,
 'global_step': 0,
 'loss': 0.3463291208109548}

In [141]:
output_eval_file = os.path.join(output_dir, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in sorted(result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

02/01/2019 10:23:16 - INFO - __main__ -   ***** Eval results *****
02/01/2019 10:23:16 - INFO - __main__ -     eval_accuracy = 0.264
02/01/2019 10:23:16 - INFO - __main__ -     eval_loss = 1.386527379353841
02/01/2019 10:23:16 - INFO - __main__ -     global_step = 0
02/01/2019 10:23:16 - INFO - __main__ -     loss = 0.34610062866441665


torch.Size([20, 4])


In [ ]:
write_predictions(eval_examples, eval_features, all_results,
                  an_best_size, max_answer_length,
                  do_lower_case, output_prediction_file,
                  output_nbest_file, output_null_log_odds_file, verbose_logging, True, null_score_diff_threshold)